# ConnorsReversal Strategy Batch Optimization

这个notebook用于对多个交易对进行ConnorsReversal策略的批量优化。

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import sys
import os

# 添加项目根目录到Python路径
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(project_root)

# 修改导入语句
from heuristic_strategy.app.plugins.connors_reversal_plugin import Plugin

In [2]:
# 配置示例
CONFIG = {
    'data_path': r'\\znas\Main\futures',
    'start_date': '2024-01-01',
    'end_date': '2024-02-01',
    'reports_path': 'reports',
    'symbols_selection': {
        'mode': 'specific',
        'specific_symbols': ['BTC', 'ETH']
    },
    'score_weights': {
        'ret_weight': 0.6,
        'sqn_weight': 0.4,
        'sharpe_weight': 0.2,
        'win_rate_weight': 0.15,
        'dd_weight': 0.1,
        'min_trades': 50
    },
    # 添加缺失的配置
    'source_timeframe': '1m',  # 源数据时间周期
    'target_timeframes': ['30min', '15min', '5min', '1min'],  # 目标时间周期
    'commission': 0.0004,
    'margin': 0.5,
    'initial_capital': 10000
}

In [3]:
class HeuristicBatchTester:
    def __init__(self, config):
        self.config = config
        self.plugin = Plugin()
        self.data_cache = {}
        
    def run_batch_test(self):
        """运行批量回测"""
        # 确保报告目录存在
        os.makedirs(self.config['reports_path'], exist_ok=True)
        
        # 获取要处理的交易对列表
        symbols = self._get_symbols()
        print(f"将回测 {len(symbols)} 个交易对")
        
        # 遍历每个交易对
        for symbol in symbols:
            self._test_single_symbol(symbol)
            
    def _test_single_symbol(self, symbol):
        """测试单个交易对"""
        print(f"\n正在处理交易对: {symbol}")
        print("=" * 50)
        
        # 使用配置中的时间周期列表
        for timeframe in self.config['target_timeframes']:
            try:
                # 准备配置
                test_config = {
                    'symbol': symbol,
                    'target_timeframe': timeframe,
                    'source_timeframe': self.config['source_timeframe'],
                    'start_date': self.config['start_date'],
                    'end_date': self.config['end_date'],
                    'data_path': self.config['data_path'],
                    'score_weights': self.config['score_weights'],
                    'commission': self.config['commission'],
                    'margin': self.config['margin'],
                    'initial_capital': self.config['initial_capital']
                }
                
                # 运行回测
                results = self._run_single_test(test_config)
                
                # 保存结果
                if results is not None:  # 添加结果检查
                    self._save_results(symbol, timeframe, results)
                
            except Exception as e:
                print(f"错误: 处理 {symbol}-{timeframe} 时发生异常: {str(e)}")
                continue
                
    def _run_single_test(self, config):
        """运行单个回测"""
        # 使用插件的评估方法
        individual = (
            self.plugin.params['lowest_point_bars'],
            self.plugin.params['rsi_length'],
            self.plugin.params['sell_barrier'],
            self.plugin.params['dca_parts']
        )
        
        # 运行评估
        score = self.plugin.evaluate_candidate(
            individual=individual,
            base_data=None,  # 这里不需要预测数据
            hourly_predictions=None,
            daily_predictions=None,
            config=config
        )
        
        return score

    def _get_symbols(self):
        """获取要处理的交易对列表"""
        if self.config['symbols_selection']['mode'] == 'specific':
            return self.config['symbols_selection']['specific_symbols']
        else:
            # 实现获取所有交易对的逻辑
            return self._get_all_symbols()
            
    def _save_results(self, symbol, timeframe, results):
        """保存回测结果"""
        results_file = os.path.join(
            self.config['reports_path'],
            f"backtest_results_{symbol}_{timeframe}.csv"
        )
        
        # 确保结果是元组形式
        if isinstance(results, tuple):
            score = results[0]
        else:
            score = results
            
        # 创建结果字典
        result_dict = {
            'Symbol': symbol,
            'Timeframe': timeframe,
            'Score': score,
            'Parameters': {
                'lowest_point_bars': self.plugin.params['lowest_point_bars'],
                'rsi_length': self.plugin.params['rsi_length'],
                'sell_barrier': self.plugin.params['sell_barrier'],
                'dca_parts': self.plugin.params['dca_parts']
            }
        }
        
        # 将结果保存为DataFrame
        df = pd.DataFrame([result_dict])
        
        # 如果文件已存在，追加结果
        if os.path.exists(results_file):
            existing_df = pd.read_csv(results_file)
            df = pd.concat([existing_df, df], ignore_index=True)
        
        df.to_csv(results_file, index=False)


In [4]:
# 使用示例
def main():
    batch_tester = HeuristicBatchTester(CONFIG)
    batch_tester.run_batch_test()

In [5]:


if __name__ == '__main__':
    main()


正在优化策略: ConnorsReversal
报告目录已创建: c:\Users\x7498\Documents\GitHub\backtrader_binance_futures\BacktestsOptimization\reports
结果文件将保存至: c:\Users\x7498\Documents\GitHub\backtrader_binance_futures\BacktestsOptimization\reports\backtest_results_ConnorsReversal.csv
发现已存在的结果文件: c:\Users\x7498\Documents\GitHub\backtrader_binance_futures\BacktestsOptimization\reports\backtest_results_ConnorsReversal.csv
将回测 2 个交易对

正在处理交易对: BTC

评估参数组合 - ConnorsReversal:
lowest_point_bars: 20
rsi_length: 10
sell_barrier: 75
dca_parts: 6

正在保存优化结果...
确保目录存在: c:\Users\x7498\Documents\GitHub\backtrader_binance_futures\BacktestsOptimization\reports
读取现有结果文件: c:\Users\x7498\Documents\GitHub\backtrader_binance_futures\BacktestsOptimization\reports\backtest_results_ConnorsReversal.csv
正在保存到CSV文件: c:\Users\x7498\Documents\GitHub\backtrader_binance_futures\BacktestsOptimization\reports\backtest_results_ConnorsReversal.csv
成功保存结果，当前共有 3 条记录

评估参数组合 - ConnorsReversal:
lowest_point_bars: 20
rsi_length: 10
sell_barrier: 75
d